In [1]:
import pandas as pd
from pathlib import Path

file = Path("__file__").parent / "data" / "un-country-data.csv"
df = pd.read_csv(file)

print()

/var/folders/rz/zcgcqm0x1bl9cj8slq9l2s1c0000gn/T/ipykernel_65764/2887683047.py:5: DtypeWarning: Columns (2,3,4,7) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(file)


In [2]:
country_df = df[df["LocTypeName"] == "Country/Area"]

In [3]:
country_df.columns

Index(['SortOrder', 'LocID', 'Notes', 'ISO3_code', 'ISO2_code', 'SDMX_code',
       'LocTypeID', 'LocTypeName', 'ParentID', 'Location', 'VarID', 'Variant',
       'Time', 'TPopulation1Jan', 'TPopulation1July', 'TPopulationMale1July',
       'TPopulationFemale1July', 'PopDensity', 'PopSexRatio', 'MedianAgePop',
       'NatChange', 'NatChangeRT', 'PopChange', 'PopGrowthRate',
       'DoublingTime', 'Births', 'Births1519', 'CBR', 'TFR', 'NRR', 'MAC',
       'SRB', 'Deaths', 'DeathsMale', 'DeathsFemale', 'CDR', 'LEx', 'LExMale',
       'LExFemale', 'LE15', 'LE15Male', 'LE15Female', 'LE65', 'LE65Male',
       'LE65Female', 'LE80', 'LE80Male', 'LE80Female', 'InfantDeaths', 'IMR',
       'LBsurvivingAge1', 'Under5Deaths', 'Q5', 'Q0040', 'Q0040Male',
       'Q0040Female', 'Q0060', 'Q0060Male', 'Q0060Female', 'Q1550',
       'Q1550Male', 'Q1550Female', 'Q1560', 'Q1560Male', 'Q1560Female',
       'NetMigrations', 'CNMR'],
      dtype='object')

In [4]:
columns = ['ISO3_code', 'Location', '1', '7', 'Time']
new_df = country_df.rename(columns={"TPopulation1Jan": "1", "TPopulation1July": "7"})

new_df = new_df[columns]

df = new_df.melt(id_vars=["ISO3_code", "Location", "Time"], var_name="Month", value_name="Population")
df['Month'] = df['Month'].astype(int)

In [5]:
df.head()

,ISO3_code,Location,Time,Month,Population
0,BDI,Burundi,1950,1,2229.322
1,BDI,Burundi,1951,1,2280.554
2,BDI,Burundi,1952,1,2330.938
3,BDI,Burundi,1953,1,2380.670
4,BDI,Burundi,1954,1,2429.703


In [6]:
df.sort_values(by=["ISO3_code", "Time", "Month"], inplace=True)
df.reset_index(drop=True, inplace=True)

In [7]:
# Add rows for populations 1 to 12 leave interpolate count between the 1 and 7 counts and then 7 and 1 counts for the next year


rows = df[['ISO3_code', 'Location', 'Time']].drop_duplicates()

dicts = []
for index, row in rows.iterrows():
    for i in range(1, 13):
        if i != 1 and i != 7:
            if row['Time'] == 2101:
                continue
            else:
                dicts.append({"ISO3_code": row["ISO3_code"], "Location": row["Location"], "Time": row["Time"], "Month": i, "Population": None})
        
merged = pd.concat((df, pd.DataFrame(dicts)))

final_df = merged.sort_values(by=["ISO3_code", "Time", "Month"])
final_df = final_df[~((final_df['Time'] == 2101) & (final_df['Month'] == 7))]

final_df.head(15)


/var/folders/rz/zcgcqm0x1bl9cj8slq9l2s1c0000gn/T/ipykernel_65764/1156808733.py:15: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  merged = pd.concat((df, pd.DataFrame(dicts)))


,ISO3_code,Location,Time,Month,Population
0,ABW,Aruba,1950,1,42.194
0,ABW,Aruba,1950,2,NaN
1,ABW,Aruba,1950,3,NaN
2,ABW,Aruba,1950,4,NaN
3,ABW,Aruba,1950,5,NaN
4,ABW,Aruba,1950,6,NaN
1,ABW,Aruba,1950,7,42.738
5,ABW,Aruba,1950,8,NaN
6,ABW,Aruba,1950,9,NaN
7,ABW,Aruba,1950,10,NaN


In [8]:
final_df['Population'] = final_df['Population'].interpolate()

In [9]:
final_df.head(15)

,ISO3_code,Location,Time,Month,Population
0,ABW,Aruba,1950,1,42.194000
0,ABW,Aruba,1950,2,42.284667
1,ABW,Aruba,1950,3,42.375333
2,ABW,Aruba,1950,4,42.466000
3,ABW,Aruba,1950,5,42.556667
4,ABW,Aruba,1950,6,42.647333
1,ABW,Aruba,1950,7,42.738000
5,ABW,Aruba,1950,8,42.828667
6,ABW,Aruba,1950,9,42.919333
7,ABW,Aruba,1950,10,43.010000


In [10]:
final_df.to_csv('test.csv', index=False)

In [ ]:
change = pd.read_csv('test.csv')
# change['Population'] = change['Population'] / 1000
change['Location'] = change['Location'].str.replace('Democratic Republic of the Congo', 'DRC')
change['Location'] = change['Location'].str.replace('United Republic of Tanzania', 'Tanzania')


change.to_csv('./data/populations-cleaned.csv', index=False)


In [36]:
change['Location'].unique()

array(['Aruba', 'Afghanistan', 'Angola', 'Anguilla', 'Albania', 'Andorra',
       'United Arab Emirates', 'Argentina', 'Armenia', 'American Samoa',
       'Antigua and Barbuda', 'Australia', 'Austria', 'Azerbaijan',
       'Burundi', 'Belgium', 'Benin', 'Bonaire, Sint Eustatius and Saba',
       'Burkina Faso', 'Bangladesh', 'Bulgaria', 'Bahrain', 'Bahamas',
       'Bosnia and Herzegovina', 'Saint Barthélemy', 'Belarus', 'Belize',
       'Bermuda', 'Bolivia (Plurinational State of)', 'Brazil',
       'Barbados', 'Brunei Darussalam', 'Bhutan', 'Botswana',
       'Central African Republic', 'Canada', 'Switzerland', 'Chile',
       'China', "Côte d'Ivoire", 'Cameroon', 'DRC', 'Congo',
       'Cook Islands', 'Colombia', 'Comoros', 'Cabo Verde', 'Costa Rica',
       'Cuba', 'Curaçao', 'Cayman Islands', 'Cyprus', 'Czechia',
       'Germany', 'Djibouti', 'Dominica', 'Denmark', 'Dominican Republic',
       'Algeria', 'Ecuador', 'Egypt', 'Eritrea', 'Western Sahara',
       'Spain', 'Estonia', '